In [ ]:
import math
import pickle
import random

import numpy as np
import pandas as pd
import torch
from IPython.display import display, clear_output
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from tqdm.notebook import tqdm

In [ ]:
interactions_df = pd.read_csv('../data/interactions_processed.csv')
users_df = pd.read_csv('../data/users_processed.csv')
items_df = pd.read_csv('../data/items_processed.csv')

In [ ]:
interactions_df.head()

In [ ]:
interactions_df = interactions_df[interactions_df['last_watch_dt'] < '2021-04-01']

In [ ]:
interactions_df.shape

In [ ]:
users_interactions_count_df = interactions_df.groupby(['user_id', 'item_id']).size().groupby(
    'user_id').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['user_id']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

In [ ]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df,
                                                            how='right',
                                                            left_on='user_id',
                                                            right_on='user_id')
print('# of interactions from users with at least 5 interactions: %d' % len(
    interactions_from_selected_users_df))

In [ ]:
def smooth_user_preference(x):
    return math.log(1 + x, 2)


interactions_full_df = interactions_from_selected_users_df \
    .groupby(['user_id', 'item_id'])['watched_pct'].sum() \
    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
interactions_full_df.head(10)

In [ ]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                                               stratify=interactions_full_df[
                                                                   'user_id'],
                                                               test_size=0.20,
                                                               random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

In [ ]:

#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('user_id')
interactions_train_indexed_df = interactions_train_df.set_index('user_id')
interactions_test_indexed_df = interactions_test_df.set_index('user_id')

In [ ]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['item_id']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [ ]:


#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100


class ModelEvaluator:

    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(interactions_full_indexed_df['item_id'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):
        try:
            index = next(i for i, c in enumerate(recommended_items) if c == item_id)
        except:
            index = -1
        hit = int(index in range(0, topn))
        return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset['item_id']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['item_id'])
        else:
            person_interacted_items_testset = {int(interacted_values_testset['item_id'])}
        interacted_items_count_testset = len(person_interacted_items_testset)

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id,
                                               items_to_ignore=get_items_interacted(person_id,
                                                                                    interactions_train_indexed_df),
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id,
                                                                               sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS,
                                                                               seed=item_id % (
                                                                                   2 ** 32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['item_id'].isin(items_to_filter_recs)]
            valid_recs = valid_recs_df['item_id'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count': hits_at_5_count,
                          'hits@10_count': hits_at_10_count,
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        for idx, person_id in enumerate(
            tqdm(list(interactions_test_indexed_df.index.unique().values))):
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)
            person_metrics['user_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
            .sort_values('interacted_count', ascending=False)

        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(
            detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(
            detailed_results_df['interacted_count'].sum())

        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}
        return global_metrics, detailed_results_df


model_evaluator = ModelEvaluator()

In [ ]:

# Constants
SEED = 42  # random seed for reproducibility
LR = 1e-3  # learning rate, controls the speed of the training
WEIGHT_DECAY = 0.01  # lambda for L2 reg. ()
NUM_EPOCHS = 200  # num training epochs (how many times each instance will be processed)
GAMMA = 0.9995  # learning rate scheduler parameter
BATCH_SIZE = 3000  # training batch size
EVAL_BATCH_SIZE = 3000  # evaluation batch size.
DEVICE = 'cuda'  #'cuda' # device to make the calculations on

In [ ]:
total_df = interactions_train_df.append(interactions_test_indexed_df.reset_index())
total_df['user_id'], users_keys = total_df.user_id.factorize()
total_df['item_id'], items_keys = total_df.item_id.factorize()

train_encoded = total_df.iloc[:len(interactions_train_df)].values
test_encoded = total_df.iloc[len(interactions_train_df):].values

In [ ]:


shape = [int(total_df['user_id'].max() + 1), int(total_df['item_id'].max() + 1)]
X_train = csr_matrix((train_encoded[:, 2], (train_encoded[:, 0], train_encoded[:, 1])),
                     shape=shape).toarray()
X_test = csr_matrix((test_encoded[:, 2], (test_encoded[:, 0], test_encoded[:, 1])),
                    shape=shape).toarray()

In [ ]:

# Initialize the DataObject, which must return an element (features vector x and target value y)
# for a given idx. This class must also have a length atribute
class UserOrientedDataset(Dataset):
    def __init__(self, X):
        super().__init__()  # to initialize the parent class
        self.X = X.astype(np.float32)
        self.len = len(X)

    def __len__(self):  # We use __func__ for implementing in-built python functions
        return self.len

    def __getitem__(self, index):
        return self.X[index]

In [ ]:

# Initialize DataLoaders - objects, which sample instances from DataObject-s
train_dl = DataLoader(
    UserOrientedDataset(X_train),
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_dl = DataLoader(
    UserOrientedDataset(X_test),
    batch_size=EVAL_BATCH_SIZE,
    shuffle=False
)

dls = {'train': train_dl, 'test': test_dl}

In [ ]:
# class Model(nn.Module):
#     def __init__(self, in_and_out_features = 8287):
#         super().__init__()
#         self.in_and_out_features = in_and_out_features
#         self.hidden_size = 500
# 
#         self.sequential = nn.Sequential( # NN architecure, where the modules modify the data sequentially
#             nn.Linear(in_and_out_features, self.hidden_size),
#             nn.ReLU(),
#             nn.Dropout(0.5),  
#             nn.Linear(self.hidden_size, self.hidden_size),
#             nn.ReLU(),
#             nn.BatchNorm1d(self.hidden_size), 
#             nn.Linear(self.hidden_size, self.hidden_size),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(self.hidden_size, in_and_out_features),
#             nn.Softmax(dim=1)
#         )
# 
#     def forward(self, x): # In the forward function, you define how your model runs, from input to output 
#         x = self.sequential(x)
#         return xb

In [57]:
class Model(nn.Module):
    def __init__(self, in_and_out_features=8287, hidden_size=500):
        super(Model, self).__init__()
        self.in_and_out_features = in_and_out_features
        self.hidden_size = hidden_size

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(in_and_out_features, 1000),
            nn.ReLU(),
            nn.Linear(1000, 500),
            nn.ReLU(),
            nn.Linear(500, hidden_size),
            nn.ReLU(),
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size, 500),
            nn.ReLU(),
            nn.Linear(500, 1000),
            nn.ReLU(),
            nn.Linear(1000, in_and_out_features),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [58]:
torch.manual_seed(SEED)  # Fix random seed to have reproducible weights of model layers

model = Model()
model.to(DEVICE)

# Initialize GD method, which will update the weights of the model
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# Initialize learning rate scheduler, which will decrease LR according to some rule
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=GAMMA)


def rmse_for_sparse(x_pred, x_true):
    mask = (x_true > 0)
    sq_diff = (x_pred * mask - x_true) ** 2
    mse = sq_diff.sum() / mask.sum()
    return mse ** (1 / 2)

In [59]:


# Training loop
metrics_dict = {
    "Epoch": [],
    "Train RMSE": [],
    "Test RMSE": [],
}

# Train loop
for epoch in tqdm(range(NUM_EPOCHS)):
    metrics_dict["Epoch"].append(epoch)
    for stage in ['train', 'test']:
        with torch.set_grad_enabled(
            stage == 'train'):  # Whether to start building a graph for a backward pass
            if stage == 'train':
                model.train()  # Enable some "special" layers (will speak about later)
            else:
                model.eval()  # Disable some "special" layers (will speak about later)

            loss_at_stage = 0
            for batch in dls[stage]:
                batch = batch.to(DEVICE)
                x_pred = model(batch)  # forward pass: model(x_batch) -> calls forward()
                loss = rmse_for_sparse(x_pred, batch)  # ¡Important! y_pred is always the first arg
                if stage == "train":
                    loss.backward()  # Calculate the gradients of all the parameters wrt loss
                    optimizer.step()  # Update the parameters
                    scheduler.step()
                    optimizer.zero_grad()  # Zero the saved gradient
                loss_at_stage += loss.item() * len(batch)
            rmse_at_stage = (loss_at_stage / len(dls[stage].dataset)) ** (1 / 2)
            metrics_dict[f"{stage.title()} RMSE"].append(rmse_at_stage)

    if (epoch == NUM_EPOCHS - 1) or epoch % 10 == 9:
        clear_output(wait=True)
        display(pd.DataFrame(metrics_dict))

,Epoch,Train RMSE,Test RMSE
0,0,2.315794,2.250791
1,1,2.087116,2.186675
2,2,1.974711,2.124670
3,3,1.899748,2.022762
4,4,1.844253,1.986536
...,...,...,...
195,195,0.980888,1.452995
196,196,0.980011,1.449011
197,197,0.977394,1.449030
198,198,0.977802,1.440107


100%|██████████| 200/200 [02:50<00:00,  1.18it/s]


In [60]:
with torch.no_grad():
    X_pred = model(torch.Tensor(X_test).to(DEVICE))
X_pred

tensor([[5.9721, 6.9679, 1.8702,  ..., 0.0000, 0.0000, 0.0000],
        [2.0620, 3.1590, 0.0702,  ..., 0.0229, 0.0000, 0.0956],
        [4.1277, 4.3343, 3.2598,  ..., 0.0000, 0.1393, 0.0317],
        ...,
        [2.3818, 3.3578, 0.2149,  ..., 0.0000, 0.0000, 0.1067],
        [2.6024, 3.5653, 3.6185,  ..., 0.0000, 0.7004, 0.3730],
        [3.8981, 4.7623, 2.3583,  ..., 0.0000, 0.1025, 0.0000]],
       device='cuda:0')

In [85]:
class AERecommender:
    MODEL_NAME = 'Autoencoder'

    def __init__(self, X_preds, X_train_and_val, X_test):
        self.X_preds = X_preds.cpu().detach().numpy()
        self.X_train_and_val = X_train_and_val
        self.X_test = X_test

    def get_model_name(self):
        return self.MODEL_NAME

    def recommend_items(self, user_id, items_to_select_idx, topn=10, verbose=False):
        user_preds = self.X_preds[user_id][items_to_select_idx]
        items_idx = items_to_select_idx[np.argsort(-user_preds)[:topn]]
        return items_idx

    def recommend(self, user_id, topn=10):
        non_interacted_items = np.argwhere(self.X_test[user_id] == 0).ravel()

        if len(non_interacted_items) == 0:
            return []
        user_preds = self.X_preds[user_id][non_interacted_items]
        items_idx = non_interacted_items[np.argsort(-user_preds)[:topn]]

        return list(items_idx)

    def evaluate(self, size=100):
        X_total = self.X_train_and_val + self.X_test

        true_5 = []
        true_10 = []

        for user_id in range(len(X_test)):
            non_zero = np.argwhere(self.X_test[user_id] > 0).ravel()
            all_nonzero = np.argwhere(X_total[user_id] > 0).ravel()
            select_from = np.setdiff1d(np.arange(X_total.shape[1]), all_nonzero)

            for non_zero_idx in non_zero:
                random_non_interacted_100_items = np.random.choice(select_from, size=20,
                                                                   replace=False)
                preds = self.recommend_items(user_id, np.append(random_non_interacted_100_items,
                                                                non_zero_idx), topn=10)
                true_5.append(non_zero_idx in preds[:5])
                true_10.append(non_zero_idx in preds)

        return {"recall@5": np.mean(true_5), "recall@10": np.mean(true_10)}


ae_recommender_model = AERecommender(X_pred, X_train, X_test)

In [86]:
ae_global_metrics = ae_recommender_model.evaluate()
ae_global_metrics

{'recall@5': 0.6073607634543179, 'recall@10': 0.8087453066332916}

In [90]:
ae_recommender_model.recommend(user_id=1, topn=10)

[7347, 6278, 3463, 3270, 3310, 7430, 2669, 3955, 4539, 3821]

In [93]:
recs = {}

for i, user_id in enumerate(interactions_full_indexed_df.index.unique()):
    user_recs = ae_recommender_model.recommend(i, 10)
    recs.update({user_id: user_recs})

In [96]:
with open('../saved_models/ae_offline.pkl', 'wb') as f:
    pickle.dump(recs, f)